In [229]:
import os
import re
import json
import pandas as pd

def extract_text_from_sheet(sheet_df):
    text_list = []
    for value in sheet_df.values.flatten():
        if pd.notna(value) and isinstance(value, str):
            # 过滤掉括号及括号中的内容
            value = value.replace('\uff08', '(').replace('\uff09', ')').replace('\uff1a', ':')
            value = value.replace("AK", "").replace("ES", "")
            value = re.sub(r'\(.*?\)', '', value)  # 使用正则表达式去除括号中的内容

            text_list.extend([text.strip() for text in value.split('\n') if text.strip()])
    return text_list

def process_excel_to_json(file_path, output_folder):
    xl = pd.ExcelFile(file_path)
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    specific_output_folder = os.path.join(output_folder, base_name)
    os.makedirs(specific_output_folder, exist_ok=True)
    
    all_text_data = {}
    for sheet_name in xl.sheet_names:
        if "Programming Details" in sheet_name:
            df = xl.parse(sheet_name)
            all_text_data["programming details"] = extract_text_from_sheet(df)
    
    if not all_text_data:
        return None
    
    json_output_path = os.path.join(specific_output_folder, 'input_data.json')
    with open(json_output_path, 'w') as json_file:
        json.dump(all_text_data, json_file, indent=4)
    
    return json_output_path

In [230]:
def process_devices(split_data, output_folder):
    devices_content = split_data.get("devices", [])
    devices_dict = {}

    current_shortname = None

    for line in devices_content:
        line = line.strip()

        # 判断是否是新的设备名称
        if line.startswith("NAME:"):
            current_shortname = line.replace("NAME:", "").strip()
            if current_shortname not in devices_dict:
                devices_dict[current_shortname] = []
            continue
        
        # 跳过 QTY 行
        if line.startswith("QTY:"):
            continue

        # 如果有当前设备名称，则添加设备
        if current_shortname:
            devices_dict[current_shortname].append(line)

    # 将字典转换为列表形式存入 devices_data
    devices_data = []
    for appearance_shortname, device_names in devices_dict.items():
        devices_data.append({
            "appearanceShortname": appearance_shortname,
            "devices": device_names
        })

    # 输出 devices.json 文件
    devices_output_path = os.path.join(output_folder, "devices.json")
    with open(devices_output_path, 'w') as file:
        json.dump({"devices": devices_data}, file, indent=4)

In [231]:
def process_groups(split_data, output_folder):
    groups_content = split_data.get("groups", [])
    groups_dict = {}

    current_group = None

    for line in groups_content:
        line = line.strip()

        # 判断是否是新的组名称
        if line.startswith("NAME:"):
            current_group = line.replace("NAME:", "").strip()
            if current_group not in groups_dict:
                groups_dict[current_group] = []
            continue
        
        # 跳过 DEVICE CONTROL: 行
        if line.startswith("DEVICE CONTROL"):
            continue

        # 如果有当前组名称，则添加设备
        if current_group:
            groups_dict[current_group].append(line)

    # 将字典转换为列表形式存入 groups_data
    groups_data = []
    for group_name, device_names in groups_dict.items():
        groups_data.append({
            "groupName": group_name,
            "devices": device_names
        })

    # 输出 groups.json 文件
    groups_output_path = os.path.join(output_folder, "groups.json")
    with open(groups_output_path, 'w') as file:
        json.dump({"groups": groups_data}, file, indent=4)

In [232]:
def parse_scene_content(scene_name, content_lines):
    contents = []
    for line in content_lines:
        parts = line.split()
        if len(parts) < 2:
            continue

        # 状态应该是倒数第二个部分
        status = parts[-1] if parts[-1] in ["ON", "OFF"] else parts[-2]
        level = 100 if status == "ON" else 0
        
        # 检查是否有级别信息
        if '+' in parts[-1]:
            try:
                level = int(parts[-1].replace("%", "").replace("+", ""))
            except (ValueError, IndexError):
                pass
        
        # 设备名称在状态之前的部分
        device_names = " ".join(parts[:-1] if parts[-1] in ["ON", "OFF"] else parts[:-2]).split(",")

        for name in device_names:
            contents.append({
                "name": name.strip(),
                "status": status,
                "statusConditions": {
                    "level": level
                }
            })
    return contents

def process_scenes(split_data, output_folder):
    scenes_content = split_data.get("scenes", [])
    scenes_data = {}
    current_scene = None

    for i, line in enumerate(scenes_content):
        line = line.strip()
        
        if line.startswith("CONTROL CONTENT:"):
            continue
        
        if line.startswith("NAME:"):
            # 这是一个新的场景名称，提取并去掉 "NAME:" 前缀
            current_scene = line.replace("NAME:", "").strip()
            if current_scene not in scenes_data:
                scenes_data[current_scene] = []
        elif current_scene:
            # 收集当前场景的设备和状态信息
            scenes_data[current_scene].extend(parse_scene_content(current_scene, [line]))

    # 转换场景数据到预期格式
    scenes_output = [{"sceneName": scene_name, "contents": contents} for scene_name, contents in scenes_data.items()]

    # 保存最终数据
    scenes_output_path = os.path.join(output_folder, "scenes.json")
    with open(scenes_output_path, 'w') as file:
        json.dump({"scenes": scenes_output}, file, indent=4)

In [233]:
def process_remote_controls(split_data, output_folder):
    remote_controls_content = split_data.get("remoteControls", [])
    remote_controls_data = []
    current_remote = None
    current_links = []

    for line in remote_controls_content:
        line = line.strip()

        # 跳过 TOTAL 行
        if line.startswith("TOTAL"):
            continue

        # 检查是否是新的远程控制器
        if line.startswith("NAME:"):
            # 如果 current_remote 已经存在，保存它的数据
            if current_remote:
                remote_controls_data.append({
                    "remoteName": current_remote,
                    "links": current_links
                })
            # 解析新的 remoteName
            current_remote = line.replace("NAME:", "").strip()
            current_links = []
        
        # 处理链接部分
        elif line.startswith("LINK:"):
            continue  # 跳过 "LINK:" 行

        # 处理每个链接项
        else:
            parts = line.split(":")
            if len(parts) < 2:
                continue

            link_index = int(parts[0].strip()) - 1  # 0-based index
            link_description = parts[1].strip()

            # 判断 action (TOGGLE, MOMENTARY) 并剥离它
            action = "normal"
            if " - " in link_description:
                link_description, action = link_description.rsplit(" - ", 1)
                action = action.strip().upper()

            # 判断链接类型
            if link_description.startswith("SCENE"):
                link_type = 2  # 场景类型
                link_name = link_description.replace("SCENE", "").strip()
            elif link_description.startswith("GROUP"):
                link_type = 1  # 组类型
                link_name = link_description.replace("GROUP", "").strip()
            elif link_description.startswith("DEVICE"):
                link_type = 0  # 设备类型
                link_name = link_description.replace("DEVICE", "").strip()
            else:
                continue  # 未知的链接类型，跳过

            current_links.append({
                "linkIndex": link_index,
                "linkType": link_type,
                "linkName": link_name,
                "action": action
            })

    # 保存最后一个远程控制器的数据
    if current_remote:
        remote_controls_data.append({
            "remoteName": current_remote,
            "links": current_links
        })

    # 输出 remoteControls.json 文件
    remote_controls_output_path = os.path.join(output_folder, "remoteControls.json")
    with open(remote_controls_output_path, 'w') as file:
        json.dump({"remoteControls": remote_controls_data}, file, indent=4)

In [234]:
def split_json_file(input_file_path, output_folder):
    with open(input_file_path, 'r') as file:
        data = json.load(file)
    content = data.get("programming details", [])
    split_keywords = {
        "devices": "KASTA DEVICE",
        "groups": "KASTA GROUP",
        "scenes": "KASTA SCENE",
        "remoteControls": "REMOTE CONTROL LINK"
    }
    split_data = {
        "devices": [],
        "groups": [],
        "scenes": [],
        "remoteControls": []
    }
    current_key = None
    for line in content:
        if line in split_keywords.values():
            current_key = next(key for key, value in split_keywords.items() if value == line)
            continue
        if current_key:
            split_data[current_key].append(line)
    os.makedirs(output_folder, exist_ok=True)
    
    # 处理设备数据
    process_devices(split_data, output_folder)
    
    # 处理组数据
    process_groups(split_data, output_folder)
    
    process_scenes(split_data, output_folder)
    
    process_remote_controls(split_data, output_folder)

def test_process_excel(input_folder, output_folder):
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.xlsx'):
            file_path = os.path.join(input_folder, file_name)
            result = process_excel_to_json(file_path, output_folder)
            if result:
                print(f"Processed {file_name} into {result}")
                split_json_file(result, os.path.dirname(result))
            else:
                print(f"No matching worksheets found in {file_name}")

# 设置输入和输出文件夹路径
input_folder = 'MPH_input'
output_folder = 'MPH_output'

# 运行测试
test_process_excel(input_folder, output_folder)

Processed Kasta programming - MPH - Type M - 20240807.xlsx into MPH_output/Kasta programming - MPH - Type M - 20240807/input_data.json
Processed Kasta programming - MPH - Type N - 20240807.xlsx into MPH_output/Kasta programming - MPH - Type N - 20240807/input_data.json
